## Cel·la de prova

ha de sortir una ruta que inclogui envs/myenv

In [2]:
import sys
print(sys.executable)

c:\Users\UX433\miniconda3\envs\myenv\python.exe


# Crear blueprint

### Comprobació preliminar del directori i kernell

Per verificar que datasets.ods i el notebook estan a la llista

In [4]:
import os, sys
print("Python executable:", sys.executable)     # mostra el Python que corre (ha de ser el env activat)
print("Directori de treball:", os.getcwd())    # carpeta del repo
print("Llistat d'arxius a la carpeta:")
for f in sorted(os.listdir('.')):
    print(" -", f)

Python executable: c:\Users\UX433\miniconda3\envs\myenv\python.exe
Directori de treball: c:\Users\UX433\OneDrive\Escritorio\networkalignment
Llistat d'arxius a la carpeta:
 - .git
 - C-el_k4_Synthetic- No groups - Simoultenously.ipynb
 - crear_blueprint.ipynb
 - datasets.ods
 - environment.yml


In [6]:
import itertools
import pickle
import matplotlib.pyplot as plt
import numpy as np
import math
import random
import os
import time
from numba import jit, njit
from numba.types import bool_, int_, float32
from math import comb
from copy import deepcopy
from tqdm import tqdm
import networkx as nx
import pandas as pd
from collections import defaultdict
from itertools import permutations

### Llegir fulls de càlcul 

In [7]:
### Open the files
tablas = 8
n_grupo = 4

d = {}
D = "Dataset"
for i in range(0,tablas):
    d["group" + str(i)] = pd.read_excel("datasets.ods", sheet_name=D+str(i+1))
    print('Read:',i, 'size', d["group" + str(i)].shape)

Read: 0 size (227, 183)
Read: 1 size (227, 183)
Read: 2 size (227, 183)
Read: 3 size (227, 183)
Read: 4 size (227, 183)
Read: 5 size (227, 183)
Read: 6 size (227, 183)
Read: 7 size (227, 183)


### Passar a matriu i binaritzem

In [8]:
## Hipermatrix M with the data (Only the synapses) 
rows = d["group" + str(i)].shape[0] - 3
columns = d["group" + str(i)].shape[1] - 3

M = np.zeros((tablas,rows,columns))
for i in range(0, tablas):
    Data = d['group' + str(i)]
    M[i,:,:] = Data.iloc[3:,3:]
    
## Since we work with same number of nodes, we want them equal and square (zeros when no connections)
M_square = np.zeros((tablas, rows, rows))
M_square[:,:, 0:columns] = M[:,:,:]

## Binarization: No weights
#Lo queremos BINARIO, ignorando su peso (Luego pensar cómo se haría con el peso)
M_square_bin = np.zeros((tablas,rows,rows))
for i in range(tablas):
    for j in range(rows):
        for k in range(columns):
            if (M[i,j,k] >= 1):
                M_square_bin[i,j,k] = 1
                
Nx = rows # Number of nodes (we imposed rows == columns)
